In [1]:
import torch
import os 
import sklearn 
import pandas as pd 
import numpy as np
from carla.data.catalog import CsvCatalog

/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
#TODO How to get access to test set from CARLA ? 
name='credit'
dataset = pd.read_csv(f'../data/{name}/{name}.csv')
            #TODO Better way for defining continous varaibles ?
continuous_wachter = dataset.drop(columns=['label']).columns
dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method='Identity')
dataset._df_test

,label,x1,x2,x3,x4,x6,x5,x7
7271,1.0,1.0,0.381805,-0.063461,7.154838,3.338277,12.391728,7.275420
1361,0.0,0.0,8.065854,-0.212944,1.405586,-0.563194,8.512333,-1.107841
8253,1.0,1.0,0.133160,-0.123246,1.089146,1.355039,-2.768613,0.195565
2597,0.0,0.0,-1.250482,-0.100334,3.406476,-0.922680,1.213075,-1.837039
4213,1.0,0.0,1.750357,-0.243642,0.124615,0.690754,1.601693,4.320406
...,...,...,...,...,...,...,...,...
7076,0.0,1.0,-5.478630,-0.102696,-1.074132,-0.274407,-2.861924,-7.894471
5999,1.0,0.0,6.962461,-0.045793,-1.200045,-0.272949,-1.110058,4.651801
3188,1.0,1.0,-1.435123,0.180970,2.918646,0.818597,2.015107,2.747512
1032,1.0,0.0,-6.171103,-0.170501,-3.956208,-3.625986,-11.473494,-3.586729


In [3]:
from string import digits
import re
model_name=['0']
accuracy=['0']
precision=['0']
recall=['0']
auc=['0']
df=pd.DataFrame([])

for name in os.listdir('./Model/'):
    #print(name.split('.')[0][-1].isdigit())
    #print(name)
    if name.split('.')[0][-1].isdigit() or name.startswith('Linear')or name.startswith('Forest'):
        full_name=name
        model=torch.load(f'./Model/{name}')
        name=name.split('_')[1]
        name=name.replace('.pth','')
        #print('FIRST',name)
        #print(name[:-1])
        if name[-1].isdigit():
            name=name[:-1]
        #print('Second',name)

        dataset = pd.read_csv(f'../data/{name}/{name}.csv')
        continuous_wachter = dataset.drop(columns=['label']).columns
        scaler='Identity'
        if name == 'economic':
            scaler='MinMax'

        dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method=scaler)
        dataset=dataset._df_test
        data_x, data_y=dataset.drop(columns=['label']), dataset['label']
        data_pred= model.predict(data_x)
        model_name.append(full_name)
        print(data_pred.shape)
        print(data_y.shape)
        if len(data_pred.shape)>1:
            data_pred= np.argmax(data_pred, axis=1)

        accuracy.append(sklearn.metrics.accuracy_score(data_y,data_pred))
        precision.append(sklearn.metrics.precision_score(data_y,data_pred))
        recall.append(sklearn.metrics.recall_score(data_y,data_pred))
        auc.append(sklearn.metrics.roc_auc_score(data_y,data_pred))
df['model_name']=model_name
df['accuracy']=accuracy
df['precision']=precision
df['recall']=recall
df['auc'] =auc
df

(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model

(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice 

,model_name,accuracy,precision,recall,auc
0,0,0,0,0,0
1,Forest_credit.pth,0.796,0.815833,0.772082,0.796349
2,Forest_economic.pth,0.8096,0.753191,0.826331,0.811696
3,Forest_sanity-3-lin.pth,0.7956,0.81664,0.780695,0.796157
4,Forest_sanity-3-non-add.pth,0.8528,0.930091,0.470046,0.728803
5,Forest_sanity-3-non-lin.pth,0.9712,0.982112,0.957641,0.970719
6,Linear_credit.pth,0.75,0.757851,0.734187,0.749987
7,Linear_economic.pth,0.8276,0.776664,0.820809,0.826615
8,Linear_sanity-3-lin.pth,0.9944,0.996154,0.993098,0.994459
9,Linear_sanity-3-non-add.pth,0.9992,0.998433,0.998433,0.998948


In [4]:
# Table to Latex 
# Get header
dataframe=pd.DataFrame([])
st=''
col=[]
name=[]
df=df.iloc[1:]
n_saver=[]
key=[]
model_name=[]
acc=[]
dataset=[]
for index, row in df.iterrows():
    #print(row['model_name'].split('_'))
    #if st is '':
    #    st += row['model_name'].split('_')[1].replace('.pth','')[:-1]
    #else:
    #    st += ' & '+row['model_name'].split('_')[1].replace('.pth','')[:-1]
    if row['model_name'].split('_')[1].replace('.pth','')[-1].isdigit():
        print('is Digit')
        data=row['model_name'].split('_')[0]+ row['model_name'].split('_')[1].replace('.pth','')[-1]
    else: 
        data=row['model_name'].split('_')[0]
    col.append(data)
    name.append(row['model_name'].split('_')[1].replace('.pth','')[:-1])
    key.append(row['model_name'])
st+= ' \\\\ '
col= np.unique(col)
name = np.unique(name[name!='credi'])
name=['economic','credit','sanity-3-lin','sanity-3-non-lin','sanity-3-non-add']
for n in name :
    st += ' & '+ n

print('col',col)
print('name',name)
print('key', key)
for i in range(0,len(col)):
    st += col[i] 
    num=''
    if col[i][-1].isdigit():
        num = col[i][-1]
    for j in range(0,len(name)):
        #print(''col[i])
        #print(name[i])
        for k in key: 
            print(f'{name[j]}{num}')
            if col[i] in k and f'{name[j]}{num}' in k: 
                t=k

        print(t)
        #print(f'{col[i]}_{name[i]}')
        #print(str(df[df['model_name'] ==f'{col[i]}_{name[j]}']['accuracy'].values[0]))
        #print(df[df['model_name'] ==f'{col[i]}_{name[i]}']['accuracy'])

        dataset.append(name[j])
        model_name.append(col[i])
        acc.append(df[df['model_name'] ==f'{t}']['accuracy'].values[0])
        st += ' & ' + str(df[df['model_name'] ==f'{t}']['accuracy'].values[0])
    st+=' \\\\ '
dataframe['model']=model_name
dataframe['dataset']=dataset
dataframe['auc']=acc
print(st)

is Digit
is Digit
is Digit
is Digit
is Digit
col ['Forest' 'Linear' 'MLP0']
name ['economic', 'credit', 'sanity-3-lin', 'sanity-3-non-lin', 'sanity-3-non-add']
key ['Forest_credit.pth', 'Forest_economic.pth', 'Forest_sanity-3-lin.pth', 'Forest_sanity-3-non-add.pth', 'Forest_sanity-3-non-lin.pth', 'Linear_credit.pth', 'Linear_economic.pth', 'Linear_sanity-3-lin.pth', 'Linear_sanity-3-non-add.pth', 'Linear_sanity-3-non-lin.pth', 'MLP_credit0.pth', 'MLP_economic0.pth', 'MLP_sanity-3-lin0.pth', 'MLP_sanity-3-non-add0.pth', 'MLP_sanity-3-non-lin0.pth']
economic
economic
economic
economic
economic
economic
economic
economic
economic
economic
economic
economic
economic
economic
economic
Forest_economic.pth
credit
credit
credit
credit
credit
credit
credit
credit
credit
credit
credit
credit
credit
credit
credit
Forest_credit.pth
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin

In [6]:
dataframe.to_csv('Classifier_results.csv')